In [3]:
import pandas as pd
import sys
from __globals import glob
from _utilities import prepare_data, prepare_database
sys.path.append("../tools/")
import _db_tools as db #Personal tool for managing sqlite databases in data science

####Helper Functions####
def limits_gen(df, means, *argv):
    '''Generate the limit values for a list containing the means in a DataFrame.
    Calculates the total mean for each fiber axis and applies it to the corresponding rows.
    Supports common (one) tolerances and single (low and high) tolerances.
    Parameters:
    df (pd.DataFrame): The MEAS dataframe to get its size.
    means (list): A list of means to generate limits for.
    *argv: Variable-length arguments for tolerances. Can be one or two values.
    Returns:
    pd.DataFrame: A DataFrame containing the generated limits.
    '''
    if len(argv) == 1: 
        l_tolerance = argv[0]
        h_tolerance = argv[0]
    elif len(argv) == 2:
        l_tolerance = argv[0]
        h_tolerance = argv[1]
    else:
        raise ValueError("Invalid number of tolerance values. Must be 1 or 2.")
    limits = pd.DataFrame(columns=["LO_LIMIT", "HI_LIMIT"])
    for index in range(int(df.shape[0])):
        if index % 2 == 0:
            low_limit = round(means[0] - h_tolerance, 4)
            high_limit = round(means[0] + l_tolerance, 4)
        else:
            low_limit = round(means[1] - h_tolerance, 4)
            high_limit = round(means[1] + l_tolerance, 4)
        current_limits_df = pd.DataFrame({"LO_LIMIT": [low_limit], "HI_LIMIT": [high_limit]}) #Create a DataFrame with the current low_limit and high_limit values
        limits = pd.concat([limits, current_limits_df], ignore_index=True, axis=0) #Concatenate the current limits DataFrame with the main 'limits' DataFrame
    return limits


In [4]:
#Data preparation
dbh = db.SQLite_Data_Extractor("database.db") #Connect to the database
MEAS = dbh.retrieve(glob.tooling) #Get the desired tooling data
dbh.close_conn() 

../database/database.db found.
Table PASSAT_B9 retrieved succesfully.
Closed connection to: ../database/database.db


In [5]:
#Calculation of the absolute means for fiber x and fiber y
resume = MEAS.transpose().describe() #Transpose the df first due to describe() working in columns.
rough_means = list(resume.iloc[1, :].values)
means = []; means_fbx = []; means_fby = [] #Preallocation
for i, mean in enumerate(rough_means): #Iterates and rounds every mean value
    mean = round(mean, 4)
    means_fbx.append(mean) if i % 2 == 0 else means_fby.append(mean)
    means.append(mean)
abs_mean_fbx = sum(means_fbx) / len(means_fbx)
abs_mean_fby = sum(means_fby) / len(means_fby)
means = [abs_mean_fbx, abs_mean_fby]
print("Means:") 
print("Fiber x: " + str(round(abs_mean_fbx, 4)))
print("Fiber y: " + str(round(abs_mean_fby, 4)))



Means:
Fiber x: 0.3313
Fiber y: 0.3555


In [6]:
LIMITS = limits_gen(MEAS, means, glob.tolerance)
LIMITS

,LO_LIMIT,HI_LIMIT
0,0.3163,0.3463
1,0.3405,0.3705
2,0.3163,0.3463
3,0.3405,0.3705
4,0.3163,0.3463
5,0.3405,0.3705
6,0.3163,0.3463
7,0.3405,0.3705
8,0.3163,0.3463
9,0.3405,0.3705


In [7]:
##Update of the database table
confirmation = input("Do you want to export the new limits to your database (y/n): ").strip().lower()
if confirmation == 'y':
    prepare_database(LIMITS, glob.tooling+"_limits") #Store a df inside the database of the project
else:
    print("Operation canceled.")

../database/database.db found.
Dataframe stored as *PASSAT_B9_limits*
../database/database.db actual contents:
    PASSAT_B9
    PASSAT_B9_limits
